In [68]:
# synpuff.ipynb content
import pandas as pd
pd.set_option("display.max_rows", None, "mode.chained_assignment", None)
from itertools import chain

import numpy as np
import datetime as dt

In [69]:
from dash import Dash, html, dcc, Input, Output, callback
import plotly.express as px
import plotly
import plotly.offline
import plotly.graph_objs as go

In [70]:
person = pd.read_csv('../../synpuff/person.csv')
condition_occurrence = pd.read_csv('../../synpuff/condition_occurrence.csv')
drug_exposure = pd.read_csv('../../synpuff/drug_exposure.csv')
concept = pd.read_csv('../../synpuff/concept.csv')

In [71]:
condition_occurrence.head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,61875376,497795,132797,2008-01-31,NaN,2008-02-07,NaN,38000200,NaN,142878.0,23882463,NaN,0389,44823969,NaN,NaN
1,137844436,1109737,133141,2008-12-12,NaN,2008-12-14,NaN,38000200,NaN,34304.0,53218870,NaN,1104,44822853,NaN,NaN
2,236827453,1906124,438699,2008-09-28,NaN,2008-10-06,NaN,38000200,NaN,83387.0,91427916,NaN,1540,44822870,NaN,NaN
3,116917366,940851,441829,2009-03-10,NaN,2009-03-17,NaN,38000200,NaN,323636.0,45136153,NaN,2760,44824089,NaN,NaN
4,171844758,1382957,437833,2009-05-24,NaN,2009-05-25,NaN,38000200,NaN,228426.0,66341079,NaN,2768,44834567,NaN,NaN


In [72]:
neoplasm_codes = [44832128,44834489,44834490,44819488,44826452,44825256]
condition_occurrence=condition_occurrence.loc[condition_occurrence['condition_source_concept_id'].isin(neoplasm_codes)]

In [73]:
concept_lookup = {c.concept_id: c.concept_name for c in concept.itertuples()}

In [74]:
def make_labels(df):
    for c in df.columns:
        if 'concept_id' in c:
            df[c.replace('_id', '_label')] = df[c].map(concept_lookup)
        if 'concept_id' in c or 'source' in c or len(df[df[c].notna()])==0:
            df = df.drop(c, axis=1)
    return df


In [75]:
person_labelled = make_labels(person)
condition_occurrence_labelled = make_labels(condition_occurrence)
drug_exposure_labelled = make_labels(drug_exposure)

In [76]:
#neoplasm_concepts = concept.loc[concept['concept_code'].isin['174.0','174.6','174.8','175.0','175.9']]
#neoplasm_concepts

In [77]:
condition_occurrence_labelled.head()

,condition_occurrence_id,person_id,condition_start_date,condition_end_date,provider_id,visit_occurrence_id,condition_concept_label,condition_type_concept_label,condition_source_concept_label,condition_status_concept_label
157,210512628,1694338,2008-09-22,2008-09-22,151287.0,81272533,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
158,10158522,81751,2009-03-22,2009-03-22,258649.0,3923058,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
159,11359134,91329,2008-09-19,2008-09-19,164692.0,4385273,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
160,251553998,2024310,2009-07-29,2009-08-03,140597.0,97112208,Malignant neoplasm of axillary tail of female ...,Outpatient header - 1st position,Malignant neoplasm of axillary tail of female ...,NaN
266,250625743,2016747,2008-11-28,2008-11-28,378143.0,96753274,Neoplasm of uncertain behavior of breast,Outpatient header - 1st position,Neoplasm of uncertain behavior of breast,NaN


,condition_occurrence_id,person_id,condition_start_date,condition_end_date,provider_id,visit_occurrence_id,condition_concept_label,condition_type_concept_label,condition_source_concept_label,condition_status_concept_label
157,210512628,1694338,2008-09-22,2008-09-22,151287.0,81272533,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
158,10158522,81751,2009-03-22,2009-03-22,258649.0,3923058,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
159,11359134,91329,2008-09-19,2008-09-19,164692.0,4385273,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
160,251553998,2024310,2009-07-29,2009-08-03,140597.0,97112208,Malignant neoplasm of axillary tail of female ...,Outpatient header - 1st position,Malignant neoplasm of axillary tail of female ...,NaN
266,250625743,2016747,2008-11-28,2008-11-28,378143.0,96753274,Neoplasm of uncertain behavior of breast,Outpatient header - 1st position,Neoplasm of uncertain behavior of breast,NaN
2251,101542215,816951,2010-05-03,2010-05-03,446058.0,39195943,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
2252,249271954,2005827,2008-04-14,2008-04-14,103492.0,96229933,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
2253,226842024,1825717,2008-06-08,2008-06-08,161379.0,87573465,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
2254,270851116,2179496,2009-09-16,2009-09-16,83797.0,104561908,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
2255,198256531,1595373,2008-03-28,2008-03-28,119174.0,76538842,Malignant neoplasm of nipple and areola of fem...,Outpatient header - 1st position,Malignant neoplasm of nipple and areola of fem...,NaN
